In [1]:
import music21 as m21
import glob
from typing import Iterator
import matplotlib.pyplot as plt
import torch

In [2]:
opuses = []
for i, abc_file_path in enumerate(glob.glob("../data/esac/*.abc")):
    print(i, end=" ")
    try:
        opuses.append(m21.converter.parse(abc_file_path))
    except Exception as ex:
        print(ex)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 Unknown type: complex
35 36 37 38 39 

In [3]:
def iter_parts(opuses: list[m21.stream.Opus]) -> Iterator[m21.stream.Part]:
    for opus in opuses:
        for score in opus[1:]:
            # yield score[0].localeOfComposition
            for part in score[1:]:
                yield part

part = next(iter_parts(opuses))

In [11]:
ts_count = 0
part_count = 0
for part in iter_parts(opuses):
    part_count += 1
    for measure in part:
        try:
            for symbol in measure: 
                if isinstance(symbol, m21.meter.TimeSignature):
                    ts_count +=1
        except TypeError:
            if isinstance(measure, m21.meter.TimeSignature):
                ts_count +=1


print(ts_count, part_count, ts_count / part_count)

13880 9789 1.4179180713045254


In [15]:
for part in iter_parts(opuses):
    count = 0
    for measure in part:
        try:
            for symbol in measure: 
                if isinstance(symbol, m21.key.Key):
                    count +=1
        except TypeError:
            break
    if count > 1:
        for measure in part:
            for symbol in measure:
                print(symbol)
            print()
        break

In [31]:
note = None
for m in part:
    print(m)
    break
    for s in m:
        if isinstance(s, m21.note.Note):
            note = s
            break
    if note:
        break
note

<music21.stream.Measure 0 offset=0.0>


In [128]:
def encode_note(note: m21.note.Note|m21.note.Rest, resolution=24) -> tuple[int, int]:
    if isinstance(note, m21.note.Rest):
        return 128, int(note.duration.quarterLength * resolution)
    if isinstance(note, m21.note.Note):
        return note.pitch.midi, int(note.duration.quarterLength * resolution)
    return None

def encode_measure(measure: m21.stream.Measure):
    return [encode_note(note) for note in measure if encode_note(note)]

def encode_part(part: m21.stream.Part):
    if isinstance(part[0], m21.stream.Measure):
        try:
            key: m21.key.Key = part[0][1]
            key.name
        except AttributeError:
            if isinstance(key, m21.key.KeySignature):
                key = key.asKey()
        return key.name, [encode_measure(m) for m in part]
    try:
        key = part[1]
        key.name
    except AttributeError:
        if isinstance(key, m21.key.KeySignature):
            key = key.asKey()
    return key.name, [encode_measure(part)]

dataset = [encode_part(part) for part in iter_parts(opuses)]

In [132]:
dataset[250][0]

'G major'

In [116]:
m21.key.Key()

<music21.key.Key of C major>

In [84]:
for sth in opuses[0]:
    if not isinstance(sth, m21.metadata.Metadata):
        
        for s in sth:
            print(s)

        break

<music21.stream.Part 0x7f563a8f6c50>


In [136]:
import yaml
import json

with open("dataset.json", "w") as file:
    json.dump(dataset, file)